In [1]:
!pip -q install chromadb sentence-transformers transformers accelerate einops --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.8 MB/s e

In [2]:
import os, time, uuid, json
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import chromadb
from chromadb.config import Settings

from sentence_transformers import SentenceTransformer

import textwrap

In [3]:
CHAT_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
EMBED_MODEL = "all-MiniLM-L6-v2"
PERSIST_DIR = "/content/memory_db"

os.makedirs(PERSIST_DIR, exist_ok=True)
print("Persist dir:", PERSIST_DIR)


Persist dir: /content/memory_db


In [4]:
class SBERTEmbeddings:
    def __init__(self, model_name: str = EMBED_MODEL):
        self.model = SentenceTransformer(model_name)
    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        vecs = self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        return [v.tolist() for v in vecs]

embedder = SBERTEmbeddings()
print("Embedding model ready:", EMBED_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model ready: all-MiniLM-L6-v2


In [5]:
client = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(anonymized_telemetry=False))
collection = client.get_or_create_collection(name="chat_memory", metadata={"hnsw:space": "cosine"})
collection

Collection(name=chat_memory)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    CHAT_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)
textgen = pipeline("text-generation", model=model, tokenizer=tokenizer)

def run_chat(messages: List[Dict[str, str]], max_new_tokens: int = 256, temperature: float = 0.2) -> str:
    """messages = [{role: system|user|assistant, content: str}, ...]"""
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    out = textgen(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=0.9, top_k=50)

    gen = out[0]["generated_text"][len(prompt):]
    return gen.strip()

print("Chat model ready:", CHAT_MODEL)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


Chat model ready: Qwen/Qwen2.5-0.5B-Instruct


In [7]:
@dataclass
class MemoryItem:
    id: str
    role: str
    content: str
    user_id: str
    conversation_id: str
    created_at: float


def add_to_memory(items: List[MemoryItem]):
    texts = [i.content for i in items]
    embeddings = embedder.embed_texts(texts)
    collection.add(
        ids=[i.id for i in items],
        documents=texts,
        metadatas=[asdict(i) for i in items],
        embeddings=embeddings,
    )


def retrieve(query: str, top_k: int = 6, where: Optional[Dict] = None):
    q_emb = embedder.embed_texts([query])[0]
    res = collection.query(
        query_embeddings=[q_emb],
        n_results=top_k,
        where=where,
        include=["documents", "metadatas", "distances"],
    )
    docs = res.get("documents", [[]])[0]
    metas = res.get("metadatas", [[]])[0]
    dists = res.get("distances", [[]])[0]
    return list(zip(docs, metas, dists))


def now_ts():
    return time.time()

USER_ID = "demo_user"
CONV_ID = str(uuid.uuid4())


In [8]:
SYS_PROMPT = (
    "You are a concise research assistant. "
    "When helpful, cite retrieved notes inline as [memory n]. If memory seems irrelevant, say so briefly before answering."
)

def build_context(retrieved, char_limit: int = 1600):
    chunks, used = [], 0
    for i, (doc, meta, dist) in enumerate(retrieved, start=1):
        s = f"[memory {i}] role={meta.get('role')} dist={dist:.3f} at {int(meta.get('created_at',0))}\n{doc.strip()}\n"
        if used + len(s) > char_limit: break
        chunks.append(s); used += len(s)
    return "\n".join(chunks) if chunks else "(no highly relevant memory found)"


def answer(user_msg: str, top_k: int = 6) -> str:
    global CONV_ID
    retrieved = retrieve(user_msg, top_k=top_k, where={"user_id": USER_ID})
    context = build_context(retrieved)
    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": f"Use the MEMORY below only if helpful. Then answer the QUESTION.\n\nMEMORY:\n{context}\n\nQUESTION: {user_msg}"},
    ]
    reply = run_chat(messages)

    add_to_memory([
        MemoryItem(str(uuid.uuid4()), "user", user_msg, USER_ID, CONV_ID, now_ts()),
        MemoryItem(str(uuid.uuid4()), "assistant", reply, USER_ID, CONV_ID, now_ts()),
    ])
    return reply

print("Agent loop ready.")

Agent loop ready.


In [15]:

seed_items = [
    MemoryItem(str(uuid.uuid4()), "user", "My name is Geek.", USER_ID, CONV_ID, now_ts()),
    MemoryItem(str(uuid.uuid4()), "assistant", "Nice to meet you, Geek!", USER_ID, CONV_ID, now_ts()),
    MemoryItem(str(uuid.uuid4()), "user", "I prefer examples in Python, not JavaScript.", USER_ID, CONV_ID, now_ts()),
]
add_to_memory(seed_items)
print("Seeded:", len(seed_items), "items")

print("\n--- Interactive Memory Assistant ---")
print("Type 'exit' to stop.\n")

while True:
    user_msg = input("You: ")
    if user_msg.lower().strip() in ["exit", "quit", "stop"]:
        print("Exiting chat...")
        break

    reply =answer(user_msg)
    print("Assistant:",reply)


Seeded: 3 items

--- Interactive Memory Assistant ---
Type 'exit' to stop.

You: give code for a simple chattbot
Assistant: Certainly! Here's a simplified version of the chatbot using `python-telegram-bot` that includes basic functionalities like command handling and text echoing.

```python
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

# Replace 'YOUR_BOT_TOKEN' with your actual bot token
TOKEN = 'YOUR_BOT_TOKEN'

def start(update, context):
    update.message.reply_text("Hello! How can I assist you today?")
    
def help_command(update, context):
    update.message.reply_text("Commands:\n/start\n/help")
    
def echo(update, context):
    update.message.reply_text(update.message.text)

def main():
    updater = Updater(TOKEN, use_context=True)
    
    dp = updater.dispatcher
    
    # Registering commands
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.command, help_command))
    
    # Start 

In [16]:

last_query = input("Enter a query to search in memory: ")

ret = retrieve(last_query, top_k=5, where={"user_id": USER_ID})

if not ret:
    print("No relevant memory found.")
else:
    print(f"\nTop {len(ret)} retrieved memories for query: '{last_query}'\n")
    for i, (doc, meta, dist) in enumerate(ret, start=1):
        ts = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(meta.get("created_at", 0)))
        print(f"[{i}] dist={dist:.3f} | role={meta['role']} | at={ts}")
        print(f"    content: {doc}\n")


Enter a query to search in memory: AI

Top 5 retrieved memories for query: 'AI'

[1] dist=0.188 | role=user | at=2025-08-19 10:32:28
    content: What is AI

[2] dist=0.316 | role=assistant | at=2025-08-19 10:32:28
    content: AI stands for "Artificial Intelligence," which refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It involves developing algorithms and systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and problem-solving. AI has numerous applications across various industries, including healthcare, finance, transportation, and more.

[3] dist=0.700 | role=user | at=2025-08-19 10:37:44
    content: Give me code a simple chatbot agent

[4] dist=0.703 | role=user | at=2025-08-19 10:32:44
    content: What is Data Science?

[5] dist=0.760 | role=assistant | at=2025-08-19 10:32:44
    content: Data science is an interdisciplinary field 

In [17]:
print("Total items in collection:", collection.count())

ret_filter = collection.query(
    query_texts=["preferences about languages"],
    n_results=3,
    where={"user_id": USER_ID},
    include=["documents", "metadatas"],
)
print("Filtered query docs:", ret_filter.get("documents", [[]])[0])

peek = collection.peek(5)
print("\nPeek IDs:", peek.get("ids"))
print("Peek metadatas (first 2):", json.dumps(peek.get("metadatas")[:2], indent=2))

import os
print("\nPersisted files:")
for root, dirs, files in os.walk(PERSIST_DIR):
    for f in files:
        print(os.path.join(root, f))

Total items in collection: 26
Filtered query docs: ['I prefer examples in Python, not JavaScript.', 'I prefer examples in Python, not JavaScript.', 'I prefer examples in Python, not JavaScript.']

Peek IDs: ['54868baf-0054-4131-b145-df82b803bd2b', 'af2178f2-8e0a-4598-9063-0d89975396d5', '48620c13-31fd-41bd-93db-99fc72c4e73a', '8771253a-cadb-4e3f-8741-2b6fbb5c9745', '6628e682-cbbd-4b69-aec9-d279293d9dca']
Peek metadatas (first 2): [
  {
    "conversation_id": "8aab6f70-e096-4562-b74f-ae7397fc875f",
    "id": "54868baf-0054-4131-b145-df82b803bd2b",
    "user_id": "demo_user",
    "content": "My name is Geek.",
    "role": "user",
    "created_at": 1755599482.3068736
  },
  {
    "content": "Nice to meet you, Geek!",
    "role": "assistant",
    "created_at": 1755599482.306888,
    "user_id": "demo_user",
    "id": "af2178f2-8e0a-4598-9063-0d89975396d5",
    "conversation_id": "8aab6f70-e096-4562-b74f-ae7397fc875f"
  }
]

Persisted files:
/content/memory_db/chroma.sqlite3
/content/memory_

In [18]:
client2 = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(anonymized_telemetry=False))
col2 = client2.get_or_create_collection(name="chat_memory")
print("Reopened count:", col2.count())
col2.query(query_texts=["What is my name?"], n_results=3)

Reopened count: 26


{'ids': [['54868baf-0054-4131-b145-df82b803bd2b',
   'cd091eb1-63e1-4dfa-9fa1-beff4a7e7edc',
   'fe906013-f05e-4303-8512-bbfc794c53bb']],
 'embeddings': None,
 'documents': [['My name is Geek.', 'My name is Geek.', 'My name is Geek.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'content': 'My name is Geek.',
    'role': 'user',
    'created_at': 1755599482.3068736,
    'id': '54868baf-0054-4131-b145-df82b803bd2b',
    'user_id': 'demo_user',
    'conversation_id': '8aab6f70-e096-4562-b74f-ae7397fc875f'},
   {'role': 'user',
    'conversation_id': '8aab6f70-e096-4562-b74f-ae7397fc875f',
    'user_id': 'demo_user',
    'content': 'My name is Geek.',
    'created_at': 1755599840.6951413,
    'id': 'cd091eb1-63e1-4dfa-9fa1-beff4a7e7edc'},
   {'content': 'My name is Geek.',
    'id': 'fe906013-f05e-4303-8512-bbfc794c53bb',
    'created_at': 1755599913.192416,
    'user_id': 'demo_user',
    'role': 'user',
    'conversation_id': '8a